In [ ]:
!pip install tensorflow tensorflow-gpu tensorflow-io matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 5.9 kB/s 
     |████████████████████████████████| 25.9 MB 1.5 MB/s 
     |████████████████████████████████| 511.7 MB 4.8 kB/s 
     |████████████████████████████████| 497.9 MB 17 kB/s 


In [ ]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio

In [ ]:
CAPUCHIN_FILE = os.path.join('data', 'Parsed_Capuchinbird_Clips', 'XC3776-3.wav')
NOT_CAPUCHIN_FILE = os.path.join('data', 'Parsed_Not_Capuchinbird_Clips', 'afternoon-birds-song-in-forest-0.wav')

In [ ]:
# Load the audio file and do some pre-processing.
def load_wav_16k_mono(filename):
  file_contents = tf.io.read_file(filename)
  wav,sample_rate = tf.audio.decode_wav(file_contents, desired_chanels = 1)
  wav = tf.squeeze(wav, axis = -1)
  sample_rate = tf.cast(sample_rate, dtype = tf.int64)
  wav = tfio.audio.resample(wav, rate_in = sample_rate, rate_out = 16000)
  return wav
wave = load_wav_16k_mono(CAPUCHIN_FILE)
nwave =  load_wav_16k_mono(NOT_CAPUCHIN_FILE) 

In [ ]:
plt.plot(wave)
plt.plot(nwave)
plt.show()

**CREATE TRAINING DATASETS:**

In [ ]:
POS = os.path.join('data', 'Parsed_Capuchinbird_Clips')
NEG = os.path.join('data', 'Parsed_Not_Capuchinbird_Clips')
pos = tf.data.Dataset.list_files(POS+'\*.wav')
neg = tf.data.Dataset.list_files(NEG+'\*.wav')

positive_ones = tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))
negative_zeros = tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))

positives = tf.data.Dataset.zip(pos, positive_ones)
negatives = tf.data.Dataset.zip(neg, negative_zeros)
train_data = positives.concatenate(negatives)

**CALCULATE THE AVERAGE LENGTH OF CLIPS:**

In [ ]:
lengths = []
for file in os.listdir(os.path.join('data', 'Parsed_Capuchinbird_Clips')):
  tensor_wave = load_wav_16k_mono(os.path.join('data', 'Parsed_Capuchinbird_Clips', file))
  lengths.append(len(tensor_wave))

tf.math.reduce_mean(lengths)
tf.math.reduce_min(lengths)
tf.math.reduce_max(lengths)

**MAIN DATA PRE-PROCESSING:**

